In [2]:
from dotenv import load_dotenv
import os
from audio_helpers import cut_audio_segment
from api_helpers import init_aai_api_key, save_raw_transcript
from transcription import extract_longest_speaker_segments

# Add FFmpeg to PATH at runtime
os.environ["PATH"] = "/usr/local/bin:" + os.environ["PATH"]

load_dotenv('.envrc')
init_aai_api_key()

Python-dotenv could not parse statement starting at line 1



# Clean background noise

There are many places that can strip background noise from your vocal recordings, with varying levels of cost-effectiveness. Here's some I've personally used:
-  [lalal.ai](www.lalal.ai) - No subscription, you can buy credits. Regular pricing sits between 5-15 processing minutes/$. There's usually massive 90% of sales around holidays to stock up on
- [elevenlabs.ai](www.elevenlabs.ai) - lowest tier is 5$/mo for 30 mins of text-to-speech. Seems like they have 60 minutes of vocal isolation for free per account: https://elevenlabs.io/app/voice-isolator


In [2]:
RAW_FILE_URL="./audio/ai_club_therapy_full_recording_raw.m4a"
CLEAN_FILE_URL="./audio/ai_club_therapy_full_recording_clean.m4a"

# [Optional] Experiment on a smaller chunk of audio

save processing credits during experimentation

In [ ]:

CLEAN_FILE_SAMPLE_URL=CLEAN_FILE_URL[:CLEAN_FILE_URL.rindex('.')] + '_shortened' + CLEAN_FILE_URL[CLEAN_FILE_URL.rindex('.'):]
cut_audio_segment(CLEAN_FILE_URL, 60, CLEAN_FILE_SAMPLE_URL)

# Transcribe with diarization

inspect transcription object for:
- confidence of words
- longest segment of continuous speech for each speaker (for good voice cloning)
- confidence of speaker
- compare confidence on a sample before and after voice cleaning


In [3]:
from api_helpers import get_transcript
transcript = get_transcript('a6d49c9c-b67c-4e04-9e2f-3ba062cd2192')

In [4]:
# Get longest segments
longest_segments = extract_longest_speaker_segments(transcript, './audio/ai_club_therapy_full_recording_clean.m4a')
longest_segments

,start,end,speaker,length
578,6380598,6454830,A,74232
392,4587204,4674820,B,87616
172,2063596,2225078,C,161482
256,3130210,3197026,D,66816


In [5]:
# write this transcription to a file
save_raw_transcript(transcript, './transcription/raw_transcript.txt')

# 4. use LLM to shorten into a 15 minute podcast script

# Example prompts
In this file you'll find transcribed dialogue between 4 speakers. Can you:
* guess the names of each speaker
* summarize the content of the talk into a few key topics
* suggest an ideal length in minutes for a condensed podcast based on the content of the conversation
* create a high level structure for the podcast reminiscent of podcasts by NYT or NPR, e.g: Planet Money that adheres roughly to the ideal length you found earlier


I like this outline a lot. You're a podcast writer for the New York Times and NPR. Can you write a script for this podcast based off this outline? You should:
* include as many direct quotes from the transcript as possible while maintaining fluidity
* maintain the characteristics and viewpoints of the original speakers. I want the personalities and perspectives to stay intact
* Try to make the overall script last between 18 and 22 minutes, as I find that as the sweet spot for podcasts for me

This is in the right direction! some edits:
* there is no host or narrator, just the four people who are talking
* include quotes around the direct quotes from the raw transcript so I know what is generated and what is real

Can you generate some more dialogue that sticks to the original outline, makes the conversation more naturally continuous, and makes sure the characters are consistent with their viewpoints from the transcript?

how long would this script last?

make sure to hit all the points in the raw transcript. aim for 15 minutes

# 5. use voice cloning from eleven labs to clone voices

In [54]:
ariel_voice_id="zjOIoGUiC56XFirKkLFQ"
merrell_voice_id="0x6lnQk4hLCTEY7oASNI"
sonia_voice_id="q1HC2axykp8y4uhDuA5Y"
prateek_voice_id="Sx55FitwWULr2NOcYsYE"

SPEAKER_NAME_TO_VOICE_ID = {
    "ARIELLE": ariel_voice_id,
    "MERYL": merrell_voice_id,
    "SONIA": sonia_voice_id,
    "PRATIK": prateek_voice_id
}

In [64]:
script_to_audio('./transcription/script.txt', './audio/test.mp3')

Generating audio for SONIA: Okay, I've been thinking about this a lot - what do you guys think is actually the point of therapy?
Generating audio for ARIELLE: It's really about relationships - "it's like a specific type of relationship that's been developed through science" where through that connection, people can understand themselves better or work through problems.
Generating audio for MERYL: Yeah, I think there are different categories though. "Like therapy when you have a really like much more severe mental health issue that needs immediate intervention" versus the more general stuff we all deal with.
Generating audio for SONIA: See, I look at it differently. "I feel like I'm in therapy to like put together puzzle pieces." It's not really about the relationship for me - it's about seeing patterns and making connections.
Generating audio for MERYL: How do your sessions usually work?
Generating audio for SONIA: "Honestly, it's pretty banal. Like somebody just says like, 'so how are